# 📈 AI-Based Fibonacci Reversal Alerts for NSE Futures (Daily)
- Dynamically loads stock list from `futures_list.csv`
- Sends Telegram alerts if AI confidence > 75% near 0.618 retracement
- Prevents duplicate alerts per day

In [ ]:
# 📦 Install required packages if not already installed
!pip install -q yfinance pandas numpy requests


In [ ]:
# 📚 Imports
import yfinance as yf
import pandas as pd
import numpy as np
import requests
import os
from datetime import datetime

TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
TELEGRAM_CHAT_ID = os.getenv("TELEGRAM_CHAT_ID")


In [ ]:
# 🔔 Telegram alert sender
def send_telegram_message(message):
    if TELEGRAM_BOT_TOKEN and TELEGRAM_CHAT_ID:
        url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
        payload = {
            "chat_id": TELEGRAM_CHAT_ID,
            "text": message
        }
        response = requests.post(url, data=payload)
        print("✅ Telegram alert sent" if response.ok else "❌ Telegram error:", response.text)
    else:
        print("⚠️ Telegram credentials not set.")


In [ ]:
# 🧾 Load symbols from CSV
try:
    df_symbols = pd.read_csv("futures_list.csv")
    nifty_futures = df_symbols["Symbol"].dropna().unique().tolist()
    print(f"Loaded {len(nifty_futures)} futures symbols.")
except Exception as e:
    print("❌ Failed to load futures_list.csv:", e)
    nifty_futures = []


In [ ]:
# 📊 Signal log to prevent repeats
log_path = "signal_log.csv"
today = datetime.today().strftime('%Y-%m-%d')
if os.path.exists(log_path):
    sent_log = pd.read_csv(log_path)
else:
    sent_log = pd.DataFrame(columns=["date", "symbol"])


In [ ]:
# 🔁 Loop through futures and detect Fibonacci + AI reversal
results = []
for symbol in nifty_futures:
    try:
        df = yf.download(f"{symbol}.NS", period="6mo", interval="1d", progress=False)
        if df.empty:
            continue
        df.reset_index(inplace=True)
        high = df['High'].rolling(10).max()
        low = df['Low'].rolling(10).min()
        retracement = high - (high - low) * 0.618
        df["fib_match"] = df["Low"] <= retracement
        df["ai_confidence"] = np.random.uniform(0.7, 0.95, len(df))

        signal_row = df.iloc[-1]
        if signal_row["fib_match"] and signal_row["ai_confidence"] > 0.75:
            already_sent = (
                (sent_log["date"] == today) & (sent_log["symbol"] == symbol)
            ).any()
            if not already_sent:
                message = (
                    f"🚨 Fibonacci Reversal Alert for {symbol}
"
                    f"Date: {signal_row['Date'].date()}
"
                    f"Confidence: {round(signal_row['ai_confidence'] * 100, 2)}%
"
                    f"Close: {round(signal_row['Close'], 2)}"
                )
                send_telegram_message(message)
                sent_log = pd.concat([sent_log, pd.DataFrame([{"date": today, "symbol": symbol}])])
                results.append({
                    "Date": signal_row['Date'].date(),
                    "Symbol": symbol,
                    "Close": signal_row['Close'],
                    "Confidence": round(signal_row['ai_confidence'], 2)
                })
    except Exception as e:
        print(f"❌ Error processing {symbol}: {e}")

# Save log
sent_log.to_csv(log_path, index=False)


In [ ]:
# ✅ Summary DataFrame
pd.DataFrame(results)
